## Pseudonym error performance under multiple interference

In [9883]:
import json
import math
import numpy as np
from scipy import signal
import h5py
import matplotlib.pyplot as plt
import scipy.signal as signal
import statistics as stats
import scipy.io
from scipy.spatial.distance import hamming
import os

rc('xtick', labelsize=14) 
rc('ytick', labelsize=14)

In [9884]:
def get_time_string(timestamp):
    '''
    Helper function to get data and time from timestamp
    INPUT: timestamp
    OUTPUT: data and time. Example: 01-04-2023, 19:50:27
    '''
    date_time = datetime.datetime.fromtimestamp(int(timestamp))
    return date_time.strftime("%m-%d-%Y, %H:%M:%S")

In [9885]:
def JsonLoad(folder, json_file):
    '''
    Load parameters from the saved json file
    INPUT
    ----
        folder: path to the measurement folder. Example: "SHOUT/Results/Shout_meas_01-04-2023_18-50-26"
        json_file: the json file with all the specifications. Example: '/save_iq_w_tx_gold.json'
    OUTPUT
    ----
        samps_per_chip: samples per chip
        wotxrepeat: number of repeating IQ sample collection w/o transmission. Used as an input to 
        traverse_dataset() func
        rxrate: sampling rate at the receiver side
    '''
    #config_file = folder+'/'+json_file
    #config_file = ""+"/"+str(folder)+"/save_iq_w_tx_file.json"
    config_dict = json.load(open(json_file))[0]
    nsamps = config_dict['nsamps']
    rxrate = config_dict['rxrate']
    rxfreq = config_dict['rxfreq']
    wotxrepeat = config_dict['wotxrepeat']
    rxrepeat = config_dict['rxrepeat']
    txnodes = config_dict['txclients']
    rxnodes = config_dict['rxclients']

    return rxrepeat, rxrate, txnodes, rxnodes

In [9886]:
def traverse_dataset(meas_folder):
    '''
    Load data from hdf5 format measurement file
    INPUT
    ----
        meas_folder: path to the measurement folder. Example: "SHOUT/Results/Shout_meas_01-04-2023_18-50-26"
    OUTPUT
    ----
        data: Collected IQ samples w/ transmission. It is indexed by the transmitter name
        noise: Collected IQ samples w/o transmission. It is indexed by the transmitter name
        txrxloc: transmitter and receiver names
    '''
    data = {}
    noise = {}
    txrxloc = {}

    dataset = h5py.File(meas_folder + '/measurements.hdf5', "r") #meas_folder
    #print("Dataset meta data:", list(dataset.attrs.items()))
    for cmd in dataset.keys():
        #print("Command:", cmd)
        if cmd == 'saveiq':
            cmd_time = list(dataset[cmd].keys())[0]
           # print("  Timestamp:", get_time_string(cmd_time))
            #print("  Command meta data:", list(dataset[cmd][cmd_time].attrs.items()))
            for rx_gain in dataset[cmd][cmd_time].keys():
               # print("   RX gain:", rx_gain)
                for rx in dataset[cmd][cmd_time][rx_gain].keys():
                    print("")
                    #print("     RX:", rx)
                    #print("       Measurement items:", list(dataset[cmd][cmd_time][rx_gain][rx].keys()))
        elif cmd == 'saveiq_w_tx':
            cmd_time = list(dataset[cmd].keys())[0]
            #print("  Timestamp:", get_time_string(cmd_time))
            #print("  Command meta data:", list(dataset[cmd][cmd_time].attrs.items()))
            for tx in dataset[cmd][cmd_time].keys():
                #print("   TX:", tx)
                
                if tx == 'wo_tx':
                    for rx_gain in dataset[cmd][cmd_time][tx].keys():
                        #print("       RX gain:", rx_gain)
                       # print(dataset[cmd][cmd_time][tx][rx_gain].keys())
                        for rx in dataset[cmd][cmd_time][tx][rx_gain].keys():
                            #print("         RX:", rx)
                            #print("           Measurement items:", list(dataset[cmd][cmd_time][tx][rx_gain][rx].keys()))
                            repeat = np.shape(dataset[cmd][cmd_time][tx][rx_gain][rx]['rxsamples'])[0]
                            #print("         repeat", repeat)

                            samplesNotx =  dataset[cmd][cmd_time][tx][rx_gain][rx]['rxsamples'][:repeat, :]
                            namelist = rx.split('-')
                            noise[namelist[1]] = samplesNotx
                else:
                    for tx_gain in dataset[cmd][cmd_time][tx].keys():
                        #print("     TX gain:", tx_gain)
                        for rx_gain in dataset[cmd][cmd_time][tx][tx_gain].keys():
                            #print("       RX gain:", rx_gain)
                            #print(dataset[cmd][cmd_time][tx][tx_gain][rx_gain].keys())
                            for rx in dataset[cmd][cmd_time][tx][tx_gain][rx_gain].keys():
                                repeat = np.shape(dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxsamples'])[0]
                                #print("         RX:", rx, "; samples shape", np.shape(dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxsamples']))
                                #print("         Measurement items:", list(dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx].keys()))
                                # print("         rxloc", (dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxloc'][0]))
                                # peak avg check
                                
                                txrxloc.setdefault(tx, []).extend([rx]*repeat)
                                rxsamples = dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxsamples'][:repeat, :]
                                data.setdefault(tx, []).append(np.array(rxsamples))

        else:                       
            print('Unsupported command: ', cmd)

    return data, noise, txrxloc

In [9887]:
def plotOnePSDForEachLink(rx_data, txrxloc, samp_rate=2e6, repeats=10):
    for txname in rx_data:
        print(txname)
        for i in range(0, len(rx_data[txname]), repeats):
            plt.figure()
            plt.psd(rx_data[txname][i][1], Fs = samp_rate/1000)
            #plt.ylim(-110, -60)
            #plt.yticks(ticks=[-110, -100, -90, -80, -70, -60])
            plt.grid('on')
            plt.title('TX: {} RX: {}'.format(txname, txrxloc[txname][i]))
            plt.xlabel('Frequency (kHz)')
            plt.tight_layout()
            plt.show()

In [9888]:
# PURPOSE: perform preamble synchronization
#          Uses the (complex-valued) preamble signal. The cross-correlation 
#          of the preamble signal and the received signal (at the time
#          when the preamble is received) should have highest magnitude
#          at the index delay where the preamble approximately starts.  
# INPUT:   rx0: received signal (with a frequency offset)
#          preambleSignal: complex, known, transmitted preamble signal 
# OUTPUT:  lagIndex: the index of rx0 where the preamble signal has highest 
#              cross-correlation
#
def crossCorrelationMax(rx0, preambleSignal):

    # Cross correlate with the preamble to find it in the noisy signal
    lags      = signal.correlation_lags(len(rx0), len(preambleSignal), mode='valid')
    xcorr_out = signal.correlate(rx0, preambleSignal, mode='valid')
    xcorr_mag = np.abs(xcorr_out)
    # Don't let it sync to the end of the packet.
    packetLenSamples = Frame_length
    maxIndex = np.argmax(xcorr_mag[:len(xcorr_mag)-packetLenSamples])
    lagIndex = lags[maxIndex]
    
#     print('The old lag ' + str(lagIndex))
    # Try to use the second lagIndex if the first one is not strong enough
    packetLenSamples = 0
    maxIndex2 = np.argmax(xcorr_mag[:len(xcorr_mag)-packetLenSamples])
    lagIndex2 = lags[maxIndex2]
    
# #     print("xcorr_mag[maxIndex2]", xcorr_mag[maxIndex2])
    
    if abs(xcorr_mag[maxIndex2]) > abs(xcorr_mag[maxIndex]):
        lagIndex = lags[maxIndex2] - Frame_length
        print('The new lag ' + str(lagIndex))
    else:
        print('The old lag ' + str(lagIndex))
    #Plot the selected signal.
    plt.figure(1)
    fig, subfigs = plt.subplots(2,1)
    subfigs[0].plot(np.real(rx0), label='Real RX Signal')
    subfigs[0].plot(np.imag(rx0), label='Imag RX Signal')
    scale_factor = np.mean(np.abs(rx0))/np.mean(np.abs(preambleSignal))
    subfigs[0].plot(range(lagIndex, lagIndex + len(preambleSignal)), scale_factor*np.real(preambleSignal), label='Preamble')
    #subfigs[0].legend()
    subfigs[1].plot(lags, xcorr_mag, label='|X-Correlation|')
    plt.xlabel('Sample Index', fontsize=14)
    plt.show()
    #plt.tight_layout()

    return lagIndex

In [9889]:
def text2bits(message):
    # Convert to characters of '1' and '0' in a vector.
    temp_message = []
    final_message = []
    for each in message:
        temp_message.append(format(ord(each), '07b'))
    for every in temp_message:
        for digit in every:
            final_message.append(int(digit))
    return final_message

In [9890]:
def binvector2str(binvector):
    #binvector = binvector[0]
    length = len(binvector)
    eps = np.finfo('float').eps
    if abs(length/7 - round(length/7)) > eps:
        print('Length of bit stream must be a multiple of 7 to convert to a string.')
    # Each character requires 7 bits in standard ASCII
    num_characters = round(length/7)
    # Maximum value is first in the vector. Otherwise would use 0:1:length-1
    start = 6
    bin_values = []
    while start >= 0:
        bin_values.append(int(math.pow(2,start)))
        start = start - 1
    bin_values = np.array(bin_values)
    bin_values = np.transpose(bin_values)
    str_out = '' # Initialize character vector
    for i in range(num_characters):
        single_char = binvector[i*7:i*7+7]
        value = 0
        for counter in range(len(single_char)):
            value = value + (int(single_char[counter]) * int(bin_values[counter]))
        str_out += chr(int(value))
    return str_out

In [9891]:
# PURPOSE: Convert binary data to M-ary by making groups of log2(M)
#          bits and converting each bit to one M-ary digit.
# INPUT: Binary digit vector, with length as a multiple of log2(M)
# OUTPUT: M-ary digit vector
def binary2mary(data, M):

    log2M   = round(np.log2(M))
    # integer number of bits per group
    if (len(data) % log2M) != 0:
        print('Input to binary2mary must be divisible by log2(m).')
    data.shape = (len(data)//log2M, log2M)
    binaryValuesArray = 2**np.arange(log2M)
    marydata = data.dot(binaryValuesArray)
    return marydata

In [9892]:
# PURPOSE: convert input data stream to signal space values for
#          a particular modulation type (as specified by the inputVec
#          and outputVec).
# INPUT: data (groups of bits)
# OUTPUT: signal space values
def lut(data, inputVec, outputVec):
    if len(inputVec) != len(outputVec):
        print('Input and Output vectors must have identical length')
    # Initialize output
    output = np.zeros(data.shape)
    # For each possible data value
    eps = np.finfo('float').eps
    for i in range(len(inputVec)):
        # Find the indices where data is equal to that input value
        for k in range(len(data)):
            if abs(data[k]-inputVec[i]) < eps:
                # Set those indices in the output to be the appropriate output value.
                output[k] = outputVec[i]
    return output

# Data Demodulation Below

In [9894]:
'''
Protocol parameters!!!
'''
FFT = 64 # FFT size for extracting IQ sample in each subcarrier
OFDM_size = 80 # OFDM symbol with cyclic prefix
data_size = 48 # data_subcarriers
mess_length = 560
Frame_length = 3040
# packet = 6000 # number of bits per pseudonym bit
# samples = packet//10 # number of samples per chip
CP = 16
repeat =10
actual_message='I have worked in SPAN Lab for the last three & half years.I implemented Pseudonymetry in POWDER.'
len(actual_message)

mess_bits = len(text2bits(actual_message))

mess_per_repeat =4

In [9895]:
CP = FFT//4  # 25% cyclic prefix
pilotValue = 2.83+2.83j # known values for pilots
pseudonymValue = 1.4142+1.4142j

In [9896]:
allCarriers = np.array([-32,-31,-30,-29,-28,-27,-26,-25,-24,-23,-22,-21,-20,-19,-18,-17,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,
                    -5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31])
dataCarriers = np.array([-26,-25,-24,-23,-22,-20,-19,-18,-17,-16,-15,-14,-13,-12,-11,-10,-9,-8,-6,
                    -5,-4,-3,-2,-1,0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,17,18,19,20,22,23,24,25,26])
pseudonymCarrier = np.array([16])
pilotCarriers = np.array([-21,-7,7,21])
guardCarriers = np.array([-32,-31,-30,-29,-28,-27,27,28,29,30,31])

In [9897]:
# Generate HTSTF signals for time synchronization
def Generate_HTSTF():
    data = scipy.io.loadmat('HTSTF.mat')
    new_data = data['stf'].reshape((1,len(data['stf'])))
    preamble = np.tile(new_data,10)[0]
    return preamble

In [9898]:
## FFT for data of window size=64
def DFT_data(x, n =64):
    return np.fft.fft(x,n)

In [9899]:
'''
calculates the distance between two arrays.
'''
def Distance(X,Y):
    count = 0
    for i in range(len(X)):
        if X[i]!= Y[i]:
            count +=1
    return count

In [9900]:
# PURPOSE: Find the symbols which are closest in the complex plane 
#          to the measured complex received signal values.
# INPUT:   Received r_hat values (output of matched filter downsampled),
#          and possible signal space complex values. 
# OUTPUT:  m-ary symbol indices in 0...length(outputVec)-1
def findClosestComplex(r_hat, outputVec):
    # outputVec is a 4-length vector for QPSK, would be M for M-QAM or M-PSK.
    # This checks, one symbol sample at a time,  which complex symbol value
    # is closest in the complex plane.
    data_out = [np.argmin(np.abs(r-outputVec)) for r in r_hat]
    return data_out

In [9901]:
# Purpose: Convert M-ary data to binary data
#          each m-ary value input in "data" is converted to
#          log2(M) binary values.
# INPUT: M-ary digit vector
# OUTPUT: Binary digit vector, with length equal to the number
#         of values in data multiplied by log2(M)
def mary2binary(data, M):
    length = len(data) # number of values in data
    log2M = round(np.log2(M)) # integer number of bits per data value
    format_string = '0' + str(log2M) + 'b'
    binarydata = np.zeros((1,length*log2M))
    count = 0
    for each in data:
        binval = format(int(each), format_string)
        for i in range(log2M):
            binarydata[0][count+i] = int(binval[i])
        count = count + log2M
    return binarydata

In [9902]:
# PURPOSE: Plot the signal symbol samples on a complex plane
# INPUT:   Received complex values (output of matched filter downsampled)
# OUTPUT:  none
def constellation_plot(rx4):
    # I like a square plot for the constellation so that both dimensions look equal
    plt.figure(figsize=(5,5))
    ax = plt.gca() 
    ax.set_aspect(1.0) # Make it a square 1x1 ratio plot
    plt.plot(np.real(rx4), np.imag(rx4),'ro')
    plt.ylabel('Imag(Symbol Sample)', fontsize=14)
    plt.xlabel('Real(Symbol Sample)', fontsize=14)
    plt.grid('on')
    plt.tight_layout()
    plt.show()

In [9903]:
def Extract_Folders(x):
    r = []
    for root, dirs, files in os.walk(x):
        r.append(dirs)
    return r

In [9904]:
def channelEstimate(x):
    pilots = x[pilotCarriers]  # extract the pilot values from the RX signal
    H_at_pilots = pilots / pilotValue # divide by the transmitted pilot values
    
    # Perform interpolation between the pilot carriers to get an estimate of the channel in the data carriers. 
    H_abs=scipy.interpolate.interp1d(pilotCarriers, abs(H_at_pilots), kind='nearest', fill_value='extrapolate')(allCarriers)
    H_phase = scipy.interpolate.interp1d(pilotCarriers, np.angle(H_at_pilots), kind='nearest', fill_value='extrapolate')(allCarriers)
    H_estimate = H_abs * np.exp(1j*H_phase)
    
    return H_estimate

In [9905]:
def equalize(OFDM_demod, Hest):
    return OFDM_demod / Hest

In [9906]:
'''PURPOSE: To recover the data signal'''
def OFDM_RX(data):
    for i in range(len(data)//(OFDM_size)):
        data_cp = data[i*(OFDM_size):(i+1)*(OFDM_size)]
        data_without_cp = data_cp[CP:]
        
        # Generate frequency domain signal
        OFDM_freq = np.fft.fft(data_without_cp,n=FFT)
        
        H_est = channelEstimate(OFDM_freq) # estimate the channel
        
        OFDM_est = equalize(OFDM_freq,H_est) # equalization with estimated channel
        
        OFDM_data = OFDM_est[dataCarriers] # extract the data signal
        if i == 0:
            OFDM_swap =  OFDM_data
        else:
            OFDM_signal = np.concatenate((OFDM_swap,  OFDM_data))
            OFDM_swap = OFDM_signal
    return OFDM_signal

In [9907]:
def Calculate_Data_BER(x):
    outputVec = np.array([1+1j, -1+1j, 1-1j, -1-1j])
    qpsk_samples = OFDM_RX(x)
    data_ber  = 0.0
    for i in range(len(x)//mess_length):
        mess_data = x[i*mess_length:(i+1)*mess_length]
        est_data = OFDM_RX(mess_data)
        # constellation_plot(est_data)
        mary_out  = findClosestComplex(est_data, outputVec)
        data_bits = mary2binary(mary_out, 4)[0]
        # print('Length of Data bits:',len(data_bits))
        print('Detected Message:',binvector2str(data_bits))
        data_ber += Distance(data_bits, text2bits(actual_message))

    return data_ber

In [9908]:
'''
Checks if pseudonyms are decoded correctly in each repeat of shout transmission.
One experiment has 10 repeats.
'''
def Average_Data_Error(x):
    # Load parameters from the JSON file which describe what was measured
 
    #folder = x
    jsonfile = "save_iq_w_tx_file.json"
    rxrepeat, samp_rate, txlocs, rxlocs = JsonLoad(x, jsonfile)
    # Load data from the HDF5 file, save IQ sample arrays
    rx_data, rx_noise, txrxloc = traverse_dataset(x)
    samp_rate = 2e6

    txloc = 'cbrssdr1-ustar-comp'
    rxloc = 'cbrssdr1-browning-comp'
    # rxloc = 'cbrssdr1-meb-comp'
    
    #Calculate SNR
    Noise = rx_noise['browning'][0]
    # Noise = rx_noise['meb'][0]
    noise_power = sum(abs(Noise)**2)/len(Noise)

    rx_data[txloc] = np.vstack(rx_data[txloc])
    rxloc_arr = np.array(txrxloc[txloc])       

    # initialize error
    data_BER = 0
    P_s = 0
    for i in range(repeat):
        repNum = i
        rx_data[txloc] = np.vstack(rx_data[txloc])
        rxloc_arr = np.array(txrxloc[txloc])
        rx0 = rx_data[txloc][rxloc_arr==rxloc][repNum]
    

        # synchronize pseudonym using preamble signal
        preambleSignal = Generate_HTSTF()
        lagIndex = crossCorrelationMax(rx0,preambleSignal)
        
        start = lagIndex + len(preambleSignal)
        Rx_signal = rx0[start:start+mess_length]
 
        data_BER +=  Calculate_Data_BER(Rx_signal)
 
        P_s += sum(abs(Rx_signal)**2)/len(Rx_signal)
    
    signal_power = P_s/repeat
    
    return data_BER, signal_power, noise_power

In [9909]:
'''
Computes the average pseudonym error in the experiment.
Experiment data is stored in folders that contain 10 repeats each.
'''
def Prob_Data_Error(x):
    folders = Extract_Folders(x)[0]
    num_folders = len(folders)
    print('Number of folders:',num_folders)
    Data_bit_BER = 0
    signal,noise = 0,0
    for i in range(num_folders):
        print(folders[i])
        error,s,n= Average_Data_Error(x + '/'+folders[i])
        
        Data_bit_BER += error
        signal += s
        noise += n

    Eb_No = 10*(np.log10(0.5*(signal-noise)/noise))

    Data_BER = Data_bit_BER/(num_folders*repeat*mess_per_repeat*mess_bits)
    
    # print("total number of pseudonym bit errors:", Data_bit_BER)
    # print('Data BER:',Data_BER)
    # print('Eb/No:',Eb_No)
    return Data_BER, Eb_No

In [9910]:
#x,y,AWGN = Prob_Pseudonym_Error("10dB_gain")
x,y = Prob_Data_Error("data")

Number of folders: 4
Shout_meas_08-17-2024_10-56-35
The new lag 1785
gb!fe0 6xg |asm |hR7e & hAlfDxeaz9nI imCIIenDu$0Pse@JmcMe|ri`in POWDER.
The new lag -55
Detected Message: I`l)z$ gm4_Ghdio S\`F XAj fex0vhe lew4)uis'a& jalf$ygiza?E"%mple/e*vpt PseMJ]cMetZ}h,n QOWDOVn
The old lag 1145
Detected Message: I(hare worked"(tan Yaf@fg10the last(thR5a & hkILESear3?I hmpmEma,dad TSe_aN_cMMvSy`lf QO~L]Fn
The new lag -695
Detected Message: I `qv`!workEd!)n$STN Lab be3 |he lasdatHRwd"0half {oasq.M aeSLA-mve'"TseulnoyMtry in POWDER.
The new lag 1225
Detected Message: I la>e ovkMd`)o STAN Lab fg: ~jw`|ast!thRga$' hAlfD{girs>I imsiI/Fu%(PseDJ]cYV3y in TM6_V.
The new lag -615
Detected Message: i l >e go$KMd !o0|af MCJ gmp tl%`lgs|`thsge& |alf$y'ars/I ioSMI+E
d%5*PseueOki%try -n PO~DGR.
The new lag 585
Detected Message: i`(yvq workedb!~0[pN Lab gg1!the laS|!thsee"0xalf yeizy.I ioAmIMntuf$P3eudOoki~2i8-n POWLER.
The new lag -535
Detected Message: i lave workMd`ho QtQN`Mcf fg2 vl% 

In [9911]:
print('SNR:',y,'\n Pseudonym_BER:',x)

SNR: 6.37381627973749 
 Pseudonym_BER: 0.022163318452380953
